In [2]:
import numpy as np
import h5py
import matplotlib.pyplot as plt
import os
import sys
sys.path.append('F:\Linux')
import illustris_python as il
import matplotlib.pyplot as plt

In [3]:
tng2il1 = np.load('F:/Linux/localRUN/Match/tng2il1_allsub.npy',allow_pickle=1).item()

#Subhalo Mophology Information
tngdisk = np.load('f:/Linux/localRUN/diskID_4WP.npy')
il1disk = np.load('f:/Linux/localRUN/diskID_il1.npy')

tng_barred = np.load('f:/Linux/localRUN/barredID_4WP_TNG.npy')
il1_barred = np.load('f:/Linux/localRUN/barredID_il1.npy')

Groups_tng = il.func.loadSubhalos('TNG', 99, "SubhaloGrNr")
Groups_il1 = il.func.loadSubhalos('il1', 135, "SubhaloGrNr")

tng_diskGIDs = Groups_tng[tngdisk]
il1_diskGIDs = Groups_il1[il1disk]

tng_barredGID = Groups_tng[tng_barred]
il1_barredGID = Groups_il1[il1_barred]

#SFR
tng_SFR = il.func.loadSubhalos('TNG', 99, "SubhaloSFRinHalfRad")
il1_SFR = il.func.loadSubhalos('il1', 135, "SubhaloSFRinHalfRad")

#Total mass
tng_mas = mas = il.func.loadSubhalos('TNG', 99, 'SubhaloMassInRadType')
il1_mas = il.func.loadSubhalos('il1', 135, 'SubhaloMassInRadType')


#Stellar mass
tng_StellarMass = np.log10(tng_mas[:,4] / 0.704 * 10**10) 
il1_StellarMass = np.log10(il1_mas[:,4] / 0.6774 * 10**10) 
tng_StellarMass[np.isnan(tng_StellarMass)] = 0
il1_StellarMass[np.isnan(il1_StellarMass)] = 0

#sSFR

#Gas Fraction
tng_GasFraction = tng_mas[:, 0] / (tng_mas[:, 4] + tng_mas[:, 0])
il1_GasFraction = il1_mas[:, 0] / (il1_mas[:, 4] + il1_mas[:, 0])
tng_GasFraction[np.isnan(tng_GasFraction)] = 0
il1_GasFraction[np.isnan(il1_GasFraction)] = 0

d:\Anaconda3\lib\site-packages\ipykernel_launcher.py:32: RuntimeWarning: invalid value encountered in true_divide
d:\Anaconda3\lib\site-packages\ipykernel_launcher.py:33: RuntimeWarning: invalid value encountered in true_divide


In [4]:
def HistValAndBin(nums, bins, more=0, mask=0):
    if mask == 1:
        reMask = []

    val = []
    tmp = nums[nums < bins[1]]
    if mask == 1:
        reMask.append(nums < bins[1])
    val.append(len(tmp))

    for i in range(1,len(bins)-1):
        tmp = nums[(nums > bins[i]) & (nums <= bins[i+1])]
        val.append(len(tmp))
        if mask == 1:
            reMask.append((nums > bins[i]) & (nums <= bins[i+1]))

    if more == 1:
        tmp = nums[nums > bins[-1]]
        val.append(len(tmp))
        if mask == 1:
            reMask.append(nums > bins[-1])

    if mask == 0:
        return np.array(val)
    else:
        return np.array(val), np.array(reMask)

def LoadMergHist(simu, subhaloID):
    '''
    return subhalo's main progenitor and merger history with snapshot
    '''
    if simu == 'TNG':
        ldir = 'f:/Linux/localRUN/tng_DiskMerTree/%d.json' % subhaloID
    else:
        ldir = 'f:/Linux/localRUN/il1_DiskMerTree/%d.json' % subhaloID
    
    with open(ldir) as f:
        data = json.load(f)
    
    Main = np.array(data['Main'])
    return dict(zip(Main[:, 0], Main[:, 1])), np.array(data['Mergers'])

def ErrorBarMedian(data):
    #return 25%, 50%, 75%
    if len(data) == 0:
        return 0, 0, 0
    elif len(data) < 3:
        return 0, np.median(data), 0
    else:
        data.sort()
        return data[int(len(data) / 4)], np.median(data), data[int(len(data) * 0.75)]


In [5]:
bar2bar = {}
for i in tng2il1.keys():
    if (i in tng_barred) and (tng2il1[i] in il1_barred):
        bar2bar[i] = tng2il1[i]

bar2nobar  = {}
for i in tng2il1.keys():
    if (i in tng_barred) and (tng2il1[i] in il1disk) and (tng2il1[i] not in il1_barred):
        bar2nobar[i] = tng2il1[i]

In [ ]:
#Plot Gas Histogram